In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import os
import sys

import pyquaternion as pyq
from math import sin, cos

sys.path.insert(1, 'H:\\minjae\\Trying Obama Dataset\\data_processing')
from tools import *

C:\Users\Wen\miniconda3\envs\tensorflow\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [3]:
N_OUTPUT = 204 # 68 x coordinate + 68 y coordinate + 68 z coordinate
WINDOW_LENGTH = 50 # in miliseconds
FEATURES = "MFCC"
N_INPUT = 26 # Number of MFCC features

In [4]:
def pad_sequence(input_vectors):
    """
    Pad array of features in order to be able to take context at each time-frame
    We pad N_CONTEXT / 2 frames before and after the signal by the features of the silence
    Args:
        input_vectors:      feature vectors for an audio

    Returns:
        new_input_vectors:  padded feature vectors
    """

    if FEATURES == "MFCC":

        # Pad sequence not with zeros but with MFCC of the silence

        silence_vectors = calculate_mfcc("data_processing/silence.wav")
        mfcc_empty_vector = silence_vectors[0]

        empty_vectors = np.array([mfcc_empty_vector] * int(N_CONTEXT / 2))

    # append N_CONTEXT/2 "empty" mfcc vectors to past
    new_input_vectors = np.append(empty_vectors, input_vectors, axis=0)
    # append N_CONTEXT/2 "empty" mfcc vectors to future
    new_input_vectors = np.append(new_input_vectors, empty_vectors, axis=0)

    return new_input_vectors

In [5]:
def create_test_sequences(dataset):
    """
    Create test sequences
    Args:
        nodes:    markers used in motion caption
        dataset:  dataset name ('train', 'test' or 'dev')

    Returns:
        nothing, saves dataset into .npy file

    """
    DATA_FILE = pd.read_csv(DATA_DIR + '/gg-'+dataset+'.csv')

    for i in range(len(DATA_FILE)):
#         input_vectors, output_vectors = create_vectors(DATA_FILE['wav_filename'][i], DATA_FILE['bvh_filename'][i])
        input_vectors, output_vectors = create_vectors(DATA_FILE['wav_filename'][i], DATA_FILE['csv_filename'][i])

        array = DATA_FILE['wav_filename'][i].split("/")
        name = array[len(array)-1].split(".")[0].split('\\')[-1]
        X = input_vectors
#         print("name:", name)
        if not os.path.isdir(DATA_DIR + '/'+dataset+'_inputs'):
            os.makedirs(DATA_DIR +  '/'+dataset+'_inputs')

        x_file_name = DATA_DIR + '/'+dataset+'_inputs/X_test_' + name + '.npy'

        np.save(x_file_name, X)

In [6]:
def create(name):
    """
    Create a dataset
    Args:
        name:  dataset: 'train' or 'test' or 'dev
        nodes: markers used in motion caption

    Returns:
        nothing: saves numpy arrays of the features and labels as .npy files

    """
    DATA_FILE = pd.read_csv(DATA_DIR + '/gg-' + str(name) + '.csv')
    X = np.array([])
    Y = np.array([])

    for i in range(len(DATA_FILE)):
        input_vectors, output_vectors = create_vectors(DATA_FILE['wav_filename'][i], DATA_FILE['csv_filename'][i])

        if len(X) == 0:
            X = input_vectors
            Y = output_vectors
        else:
            X = np.concatenate((X, input_vectors), axis=0)
            Y = np.concatenate((Y, output_vectors), axis=0)

        if i%3==0:
            print("^^^^^^^^^^^^^^^^^^")
            print('{:.2f}% of processing for {:.8} dataset is done'.format(100.0 * (i+1) / len(DATA_FILE), str(name)))
            print("Current dataset sizes are:")
            print(X.shape)
            print(Y.shape)

    x_file_name = DATA_DIR + '/X_' + str(name) + '.npy'
    y_file_name = DATA_DIR + '/Y_' + str(name) + '.npy'
    np.save(x_file_name, X)
    np.save(y_file_name, Y)


In [7]:
import re
def create_vectors(audio_filename, gesture_filename):
    """
    Extract features from a given pair of audio and motion files
    Args:
        audio_filename:    file name for an audio file (.wav)
        gesture_filename:  file name for a motion file (.csv)
        nodes:             an array of markers for the motion

    Returns:
        input_with_context   : speech features
        output_with_context  : motion features
    """
    # Step 1: Vactorizing speech, with features of N_INPUT dimension, time steps of 0.01s
    # and window length with 0.025s => results in an array of 100 x N_INPUT
    AUs = [1, 2, 4, 5, 6, 7, 9, 10, 12, 14, 15, 17, 20, 23, 25, 26, 45] # 17 features
    
    
    if FEATURES == "MFCC":

        input_vectors = calculate_mfcc(audio_filename)
#         print("input_vectors:",input_vectors)
#         print(len(input_vectors))
#         print("shape of input_vectors:", input_vectors.shape)

    
    # Step 2: Read motions

    motion_format = "csv"

    if motion_format == "csv":
        df = pd.read_csv(gesture_filename, delimiter=',', encoding="utf-8")
        frametime = df.shape[0]
        
        #downsampled to 20FPS from 30FPS
        df = df.drop(df.index[2::3])
        df = df.reset_index(drop=True)
        
#         print("df[0]:", df.iloc[0])
#         output_vectors = np.array([[],[]])
        length_df = frametime
        output_vectors = np.zeros(shape=(length_df,N_OUTPUT))
        column_regex_pattern = re.compile(r'^ [XYZ]_[0-9]*')
#         print('x_regex_pattern:',x_regex_pattern)
        columns = df.columns[df.columns.str.contains(column_regex_pattern)]
        output_vectors = df[columns].values
            
    
    
    # Step 3: Align vector length
    input_vectors, output_vectors = shorten(input_vectors, output_vectors)

    
    # Step 4: Retrieve N_CONTEXT each time, stride one by one
    input_with_context = np.array([])
    output_with_context = np.array([])

    strides = len(input_vectors)

    input_vectors = pad_sequence(input_vectors)
    
#     print("input_vectors_after_padding:", input_vectors)
#     print("shape of input_vectors_after_padding:", input_vectors.shape)
    
    for i in range(strides):
        stride = i + int(N_CONTEXT/2)
        if i == 0:
            input_with_context = input_vectors[stride - int(N_CONTEXT/2) : stride + int(N_CONTEXT/2) + 1].reshape(1, N_CONTEXT+1, N_INPUT)
            output_with_context = output_vectors[i].reshape(1, N_OUTPUT)
        else:
            input_with_context = np.append(input_with_context, input_vectors[stride - int(N_CONTEXT/2) : stride + int(N_CONTEXT/2) + 1].reshape(1, N_CONTEXT+1, N_INPUT), axis=0)
            output_with_context = np.append(output_with_context, output_vectors[i].reshape(1, N_OUTPUT), axis=0)
    
    return input_with_context, output_with_context

In [8]:
DATA_DIR = 'data_3d_cord'
N_CONTEXT = 60


In [9]:
create('test')
create('dev')
create('train')

KeyboardInterrupt: 

In [10]:
create_test_sequences('test')

In [18]:
DATA_FILE = pd.read_csv(DATA_DIR + '/gg-' + str('train') + '.csv')

In [19]:
DATA_FILE

,wav_filename,wav_filesize,csv_filename
0,H:\minjae\Trying Obama Dataset\data\train\inpu...,14409806,H:\minjae\Trying Obama Dataset\data\train\labe...
1,H:\minjae\Trying Obama Dataset\data\train\inpu...,16961614,H:\minjae\Trying Obama Dataset\data\train\labe...
2,H:\minjae\Trying Obama Dataset\data\train\inpu...,14594126,H:\minjae\Trying Obama Dataset\data\train\labe...
3,H:\minjae\Trying Obama Dataset\data\train\inpu...,15577166,H:\minjae\Trying Obama Dataset\data\train\labe...
4,H:\minjae\Trying Obama Dataset\data\train\inpu...,4042830,H:\minjae\Trying Obama Dataset\data\train\labe...
5,H:\minjae\Trying Obama Dataset\data\train\inpu...,13533262,H:\minjae\Trying Obama Dataset\data\train\labe...
6,H:\minjae\Trying Obama Dataset\data\train\inpu...,8552526,H:\minjae\Trying Obama Dataset\data\train\labe...
7,H:\minjae\Trying Obama Dataset\data\train\inpu...,12877902,H:\minjae\Trying Obama Dataset\data\train\labe...
8,H:\minjae\Trying Obama Dataset\data\train\inpu...,6660174,H:\minjae\Trying Obama Dataset\data\train\labe...
9,H:\minjae\Trying Obama Dataset\data\train\inpu...,19148878,H:\minjae\Trying Obama Dataset\data\train\labe...


In [29]:
df = pd.read_csv('data', delimiter=',', encoding="utf-8")

PermissionError: [Errno 13] Permission denied: 'data'

In [31]:
df = pd.read_csv('data/train/labels/pose1.csv', delimiter=',', encoding="utf-8")

In [48]:
x_regex_pattern = re.compile(r'^ x_[0-9]*')
y_regex_pattern = re.compile(r'^ [xy]_[0-9]*')
print('x_regex_pattern:',x_regex_pattern)
x_columns = df.columns[df.columns.str.contains(x_regex_pattern)]
y_columns = df.columns[df.columns.str.contains(y_regex_pattern)]
print("x_columns:", x_columns)
print("y_columns:", y_columns)

x_regex_pattern: re.compile('^ x_[0-9]*')
x_columns: Index([' x_0', ' x_1', ' x_2', ' x_3', ' x_4', ' x_5', ' x_6', ' x_7', ' x_8',
       ' x_9', ' x_10', ' x_11', ' x_12', ' x_13', ' x_14', ' x_15', ' x_16',
       ' x_17', ' x_18', ' x_19', ' x_20', ' x_21', ' x_22', ' x_23', ' x_24',
       ' x_25', ' x_26', ' x_27', ' x_28', ' x_29', ' x_30', ' x_31', ' x_32',
       ' x_33', ' x_34', ' x_35', ' x_36', ' x_37', ' x_38', ' x_39', ' x_40',
       ' x_41', ' x_42', ' x_43', ' x_44', ' x_45', ' x_46', ' x_47', ' x_48',
       ' x_49', ' x_50', ' x_51', ' x_52', ' x_53', ' x_54', ' x_55', ' x_56',
       ' x_57', ' x_58', ' x_59', ' x_60', ' x_61', ' x_62', ' x_63', ' x_64',
       ' x_65', ' x_66', ' x_67'],
      dtype='object')
y_columns: Index([' x_0', ' x_1', ' x_2', ' x_3', ' x_4', ' x_5', ' x_6', ' x_7', ' x_8',
       ' x_9',
       ...
       ' y_58', ' y_59', ' y_60', ' y_61', ' y_62', ' y_63', ' y_64', ' y_65',
       ' y_66', ' y_67'],
      dtype='object', length=136)


In [39]:
df[x_columns].loc[0]

 x_0     369.0
 x_1     371.7
 x_2     376.7
 x_3     383.9
 x_4     394.6
         ...  
 x_63    472.8
 x_64    493.9
 x_65    474.1
 x_66    463.5
 x_67    453.7
Name: 0, Length: 68, dtype: float64

In [45]:
df[x_columns]

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,...,x_58,x_59,x_60,x_61,x_62,x_63,x_64,x_65,x_66,x_67
0,369.0,371.7,376.7,383.9,394.6,409.5,425.2,444.7,466.4,486.9,...,455.1,441.1,433.6,453.0,462.4,472.8,493.9,474.1,463.5,453.7
1,369.2,372.0,377.1,384.4,395.4,410.5,426.1,445.3,466.7,487.1,...,455.0,440.9,433.3,453.0,462.4,472.7,494.3,473.9,463.4,453.7
2,369.0,371.9,377.2,384.5,395.4,410.4,426.1,445.6,467.1,487.5,...,454.8,440.8,433.3,452.8,462.3,472.7,494.4,474.0,463.4,453.6
3,369.1,371.8,377.0,384.3,395.1,410.0,425.9,445.5,467.1,487.6,...,454.6,440.5,433.0,452.6,462.2,472.8,494.5,474.0,463.2,453.3
4,369.1,371.8,376.9,384.1,394.9,409.9,425.6,445.1,466.6,487.1,...,454.4,440.3,432.8,452.5,462.0,472.6,494.3,473.7,463.0,453.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2244,393.0,389.7,388.8,390.7,397.3,408.3,421.0,439.4,461.3,482.3,...,464.4,450.3,442.5,467.3,477.5,488.8,506.6,487.4,476.0,465.5
2245,392.5,389.2,388.4,390.4,397.0,407.9,420.7,439.0,460.9,481.8,...,464.2,450.2,442.2,466.7,476.8,488.1,506.0,486.9,475.6,465.2
2246,392.3,389.1,388.3,390.2,396.9,407.8,420.6,438.9,460.8,481.7,...,464.2,450.2,442.2,466.8,476.8,488.0,505.9,486.8,475.6,465.3
2247,391.8,388.6,387.8,389.8,396.3,407.3,420.1,438.5,460.5,481.6,...,463.3,449.5,441.5,465.7,475.7,486.9,504.9,485.9,474.7,464.4


In [60]:
df[y_columns]

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,...,y_58,y_59,y_60,y_61,y_62,y_63,y_64,y_65,y_66,y_67
0,369.0,371.7,376.7,383.9,394.6,409.5,425.2,444.7,466.4,486.9,...,288.1,283.9,271.3,264.0,263.8,261.7,264.2,273.6,276.0,275.9
1,369.2,372.0,377.1,384.4,395.4,410.5,426.1,445.3,466.7,487.1,...,287.7,283.7,271.4,264.2,264.0,261.8,264.1,273.2,275.7,275.6
2,369.0,371.9,377.2,384.5,395.4,410.4,426.1,445.6,467.1,487.5,...,288.0,283.8,271.4,264.1,263.8,261.6,264.1,273.5,276.0,275.8
3,369.1,371.8,377.0,384.3,395.1,410.0,425.9,445.5,467.1,487.6,...,287.6,283.6,271.4,264.2,264.0,261.7,264.1,273.1,275.6,275.4
4,369.1,371.8,376.9,384.1,394.9,409.9,425.6,445.1,466.6,487.1,...,287.6,283.5,271.4,264.4,264.2,262.0,264.2,273.0,275.5,275.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2244,393.0,389.7,388.8,390.7,397.3,408.3,421.0,439.4,461.3,482.3,...,288.9,282.2,269.7,268.2,270.4,270.6,277.1,280.4,280.3,277.8
2245,392.5,389.2,388.4,390.4,397.0,407.9,420.7,439.0,460.9,481.8,...,288.6,282.2,270.4,269.1,271.3,271.4,277.5,279.9,279.8,277.4
2246,392.3,389.1,388.3,390.2,396.9,407.8,420.6,438.9,460.8,481.7,...,288.6,282.2,270.4,269.1,271.3,271.4,277.5,279.8,279.8,277.3
2247,391.8,388.6,387.8,389.8,396.3,407.3,420.1,438.5,460.5,481.6,...,287.7,281.7,270.6,269.6,271.7,271.8,277.7,279.0,279.0,276.5


In [52]:
len(df)

2249

In [53]:
output_vectors = np.zeros(shape=(len(df),N_OUTPUT))

In [58]:
output_vectors = df[y_columns].values

In [59]:
output_vectors

array([[369. , 371.7, 376.7, ..., 273.6, 276. , 275.9],
       [369.2, 372. , 377.1, ..., 273.2, 275.7, 275.6],
       [369. , 371.9, 377.2, ..., 273.5, 276. , 275.8],
       ...,
       [392.3, 389.1, 388.3, ..., 279.8, 279.8, 277.3],
       [391.8, 388.6, 387.8, ..., 279. , 279. , 276.5],
       [391. , 387.9, 387.3, ..., 278.6, 278.6, 276.1]])